In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
!pip install pyclustering
!pip install scikit-learn-extra

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
interventions = pd.read_csv('/content/sample_data/intervention_without_aed_hospital.csv',encoding='utf-8')
interventions[(interventions['lat'].isna()) | (interventions['lon'].isna())]

,mission_id,t0_date,duration_t3-t0,eventtype_code,eventlevel,province,postalcode_intervention,lat,lon,abandon_reason


In [ ]:
interventions['province'].unique()

array(['Province du Brabant flamand', 'Province de Limbourg', nan,
       'Province d’Anvers', 'Province de Flandre orientale',
       'Province de Flandre occidentale', 'Province de Hainaut',
       'Région de Bruxelles-Capitale', 'Province du Brabant wallon',
       'False', 'Province de Namur', 'Province de Luxembourg'],
      dtype=object)

In [ ]:
interventions = interventions[-interventions['lat'].isna()]
interventions = interventions[-interventions['lon'].isna()]

In [ ]:
interventions_bra = interventions[interventions['province']=='Province du Brabant flamand']
interventions_bra

,mission_id,t0_date,duration_t3-t0,eventtype_code,eventlevel,province,postalcode_intervention,lat,lon,abandon_reason
0,21221530192,2022-07-15,NaN,P003,N1,Province du Brabant flamand,NaN,50.928019,4.535123,NaN
1,21221530192,2022-07-15,NaN,P003,N1,Province du Brabant flamand,NaN,50.928019,4.535123,NaN
2,21221530420,2022-06-02,9.833333,P003,N1,Province du Brabant flamand,NaN,50.767029,4.273000,NaN
3,21221530420,2022-06-02,9.833333,P003,N1,Province du Brabant flamand,NaN,50.767029,4.273000,NaN
4,21221540122,2022-06-03,15.050000,P003,N1,Province du Brabant flamand,NaN,50.898349,4.755029,NaN
...,...,...,...,...,...,...,...,...,...,...
3695,70222130192,2022-08-01,NaN,P003,N1,Province du Brabant flamand,NaN,50.916690,5.068760,NaN
5632,22223120052,2022-11-08,13.666983,P003,N1,Province du Brabant flamand,NaN,50.753580,4.955580,NaN
5633,22223120052,2022-11-08,21.950750,P003,N1,Province du Brabant flamand,NaN,50.753580,4.955580,NaN
5700,22230750078,2023-03-16,9.483133,P003,N1,Province du Brabant flamand,NaN,50.715910,4.108900,Overleden


In [ ]:
interventions_bra[(interventions_bra['latitude_intervention'].isna()) | (interventions_bra['longitude_intervention'].isna())]

,mission_id,service_name,latitude_permanence,longitude_permanence,permanence_short_name,permanence_long_name,vector_type,cityname_intervention,latitude_intervention,longitude_intervention,...,cityname_destination_hospital,streetname_destination_hospital,housenumber_destination_hospital,calculated_traveltime_destinatio,calculated_distance_destination,number_of_transported_persons,abandon_reason,eventtype,eventlevel,province


In [ ]:
latlon = interventions_bra.loc[:,['lat','lon']]
increasepercentage = 0.1
newaed_num = 1230*increasepercentage

In [ ]:
latlon

,lat,lon
0,50.928019,4.535123
1,50.928019,4.535123
2,50.767029,4.273000
3,50.767029,4.273000
4,50.898349,4.755029
...,...,...
3695,50.916690,5.068760
5632,50.753580,4.955580
5633,50.753580,4.955580
5700,50.715910,4.108900


In [ ]:
from sklearn_extra.cluster import KMedoids
kmedoids = KMedoids(n_clusters= 246, random_state=0, metric='haversine').fit(latlon)
newaed_loc = kmedoids.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 3 is empty! self.labels_[self.medoid_indices_[3]] may not be labeled with its corresponding cluster (3).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 6 is empty! self.labels_[self.medoid_indices_[6]] may not be labeled with its corresponding cluster (6).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 7 is empty! self.labels_[self.medoid_indices_[7]] may not be labeled with its corresponding cluster (7).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 9 is empty! self.labels_[self.medoid_indices_[9]] may not be labeled with its corresponding cluster (9).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 11

In [ ]:
newaed_loc

array([[50.897429,  4.480603],
       [50.928019,  4.535123],
       [50.916193,  4.507016],
       [50.916193,  4.507016],
       [50.897429,  4.480603],
       [50.879894,  4.472799],
       [50.879894,  4.472799],
       [50.897429,  4.480603],
       [50.897429,  4.480603],
       [50.91543 ,  4.508336],
       [50.897429,  4.480603],
       [50.897429,  4.480603],
       [50.897429,  4.480603],
       [50.897429,  4.480603],
       [50.8811  ,  4.469291],
       [50.879272,  4.498266],
       [50.879272,  4.498266],
       [50.91543 ,  4.508336],
       [50.909013,  4.513117],
       [50.909013,  4.513117],
       [50.897429,  4.480603],
       [50.890807,  4.465508],
       [50.890807,  4.465508],
       [50.88157 ,  4.475748],
       [50.88157 ,  4.475748],
       [50.91479 ,  4.507402],
       [50.91479 ,  4.507402],
       [50.897429,  4.480603],
       [50.897429,  4.480603],
       [50.897429,  4.480603],
       [50.921852,  4.513904],
       [50.896042,  4.439419],
       [

In [ ]:
list(newaed_loc[:,0])

[50.897429,
 50.928019,
 50.916193,
 50.916193,
 50.897429,
 50.879894,
 50.879894,
 50.897429,
 50.897429,
 50.91543,
 50.897429,
 50.897429,
 50.897429,
 50.897429,
 50.8811,
 50.879272,
 50.879272,
 50.91543,
 50.909013,
 50.909013,
 50.897429,
 50.890807,
 50.890807,
 50.88157,
 50.88157,
 50.91479,
 50.91479,
 50.897429,
 50.897429,
 50.897429,
 50.921852,
 50.896042,
 50.924917,
 50.924917,
 50.896042,
 50.893465,
 50.841788,
 50.907779,
 50.98498,
 50.865579,
 50.865579,
 50.895547,
 50.895547,
 50.97234,
 50.907331,
 50.907779,
 50.894559,
 50.894559,
 50.892886,
 50.933812,
 50.947779,
 50.883454,
 50.97098,
 50.858384,
 50.816022,
 50.844546,
 50.858384,
 50.861121,
 50.8494,
 50.781567,
 50.765019,
 50.823793]

In [ ]:
pd.DataFrame(dict(clustered_lat=list(newaed_loc[:,0]),clustered_lon=list(newaed_loc[:,1])))

,clustered_lat,clustered_lon
0,50.897429,4.480603
1,50.928019,4.535123
2,50.916193,4.507016
3,50.916193,4.507016
4,50.897429,4.480603
...,...,...
57,50.861121,4.513992
58,50.849400,4.505767
59,50.781567,4.289252
60,50.765019,4.490417


In [ ]:
newaedloc_Brabant_flamand_increase10per = pd.DataFrame(dict(clustered_lat=list(newaed_loc[:,0]),clustered_lon=list(newaed_loc[:,1])))

In [ ]:
newaedloc_Brabant_flamand_increase5per = pd.DataFrame(dict(clustered_lat=list(newaed_loc[:,0]),clustered_lon=list(newaed_loc[:,1])))

In [ ]:
newaedloc_Brabant_flamand_increase10per.to_csv('/content/drive/MyDrive/Datathon_Data/Processed/newaedloc_Brabant_flamand_increase10per_carelated.csv',index=False)

In [ ]:
newaedloc_Brabant_flamand_increase5per.to_csv('/content/drive/MyDrive/Datathon_Data/Processed/newaedloc_Brabant_flamand_increase5per_carelated.csv',index=False)

In [ ]:
!pip install folium

In [ ]:
import folium

newlatlon = generate_candidate_sites(latlon,1000)

# 创建地图对象
m = folium.Map()

# 添加标记
for lat,lon in latlon:
    folium.Marker(location=[lat, lon], popup='Latitude: {}, Longitude: {}'.format(lat, lon)).add_to(m)
for lat, lon in newlatlon:
    folium.Marker(location=[lat, lon], popup='Latitude: {}, Longitude: {}'.format(lat, lon), icon=folium.Icon(color='red')).add_to(m)
# 显示地图
m

In [ ]:
newlatlon

NameError: name 'newlatlon' is not defined